In [7]:
#!/usr/bin/env python

import tensorflow as tf
# set hardware config
#tf.debugging.set_log_device_placement(True)

cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')

# set gpu memory grouth automatically
#for gpu in gpus:
#    tf.config.experimental.set_memory_growth(gpu, True)

if(gpus!=[]):
    # set virtal gpu/ logical gpu, create four logical gpu from a physical gpu (gpus[0])
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3072),
        tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3072),
        tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3072),
        tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3072)
        ]
        )

logical_cpus = tf.config.experimental.list_logical_devices(device_type='CPU')
logical_gpus = tf.config.experimental.list_logical_devices(device_type='GPU')
print('physical cpus and gpus: ',cpus, gpus)
print('physical cpus number: ', len(cpus))
print('physical gpus number: ', len(gpus))
print('logical cpus and gpus: ',logical_cpus, logical_gpus)
print('logical cpgs number: ', len(logical_gpus))

print("tensorflow version:",tf.__version__)

import os
import sys
sys.path.append("./../")
import numpy as np
import matplotlib.pyplot as plt
import pdb
import pandas as pd
import yaml
import h5py
import vicon_imu_data_process.process_rawdata as pro_rd
import estimation_assessment.scores as es_as

import seaborn as sns
import copy
import re
import json

from vicon_imu_data_process.const import FEATURES_FIELDS, LABELS_FIELDS, DATA_PATH, TRAIN_USED_TRIALS
from vicon_imu_data_process.const import DROPLANDING_PERIOD, RESULTS_PATH
from vicon_imu_data_process import const
from vicon_imu_data_process.dataset import *

from estimation_models.rnn_models import *
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
import time as localtimepkg

from estimation_study import *
from estimation_assessment.visualization import *
#import pdb
import re


physical cpus and gpus:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')] []
physical cpus number:  1
physical gpus number:  0
logical cpus and gpus:  [LogicalDevice(name='/device:CPU:0', device_type='CPU')] []
logical cpgs number:  0
tensorflow version: 2.5.0
tensorflow version: 2.5.0
tensorflow version: 2.5.0
[LogicalDevice(name='/device:CPU:0', device_type='CPU')] []


In [60]:
    
def lstm_part(inputs,hyperparams):
    x = tf.keras.layers.LSTM(int(hyperparams['lstm_units']),
                           return_sequences=True, 
                           activation='tanh',
                           input_shape=[None,int(hyperparams['features_num'])])(inputs)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(units=60, activation=tf.nn.relu)(x)
    x = tf.keras.layers.Dropout(0.2)(x) 
    x = tf.keras.layers.Dense(units=30, activation=tf.nn.relu)(x)
    outputs = tf.keras.layers.Dense(units=int(hyperparams['labels_num']))(x)
    return outputs
    
def fix_part(inputs):
    if(inputs<10):
        return tf.zeros(shape=[1,1])
    else:
        return tf.ones(shape=[1,1])

def model_v2(hyperparams):
    input_a = tf.keras.Input(shape=(None,int(hyperparams['features_num'])))
    input_b = tf.keras.Input(shape=(None,int(hyperparams['labels_num'])))
    y1 = lstm_part(input_a,hyperparams)
    y2 = tf.keras.layers.Lambda(fix_part,
                                input_shape=[None,int(hyperparams['labels_num'])],
                                output_shape=[None,int(hyperparams['labels_num'])])
    outputs = tf.keras.layers.Multiply()([y1,y2])
    model = tf.keras.Model(inputs=[input_a, input_b], outputs=outputs)
    return model

hyperparams = initParameters()
hyperparams['lstm_units']=5

model = model_v2(hyperparams)

TypeError: 'NoneType' object is not subscriptable

In [42]:
y2 = tf.keras.layers.Lambda(fix_part,
                                input_shape=[None,int(hyperparams['labels_num'])],
                                output_shape=[None,int(hyperparams['labels_num'])])

In [94]:
input_a = tf.keras.Input(shape=(None,int(hyperparams['features_num'])))
input_b = tf.keras.Input(shape=(None,int(hyperparams['labels_num'])))
print('input_a:',input_b)
print('input_b:',input_b)
y1 = lstm_part(input_a,hyperparams)
y2 = tf.keras.layers.Lambda(fix_part,
                                input_shape=[None,int(hyperparams['labels_num'])],
                                output_shape=[None,int(hyperparams['labels_num'])])
#y2 = tf.vari([[0,0,0,0,0,0,0,0, 1,1,1,1,1]],dtype=tf.float32)
y2 = tf.zeros(shape=[1,1]) if False else tf.ones(shape=[1,1])
print('y1:',y1)
print('y2:',y2)

outputs = tf.keras.layers.Multiply()([y1,y2])
print('outputs: ',outputs)
model = tf.keras.Model(inputs=[input_a, input_b], outputs=outputs)


input_a: KerasTensor(type_spec=TensorSpec(shape=(None, None, 1), dtype=tf.float32, name='input_136'), name='input_136', description="created by layer 'input_136'")
input_b: KerasTensor(type_spec=TensorSpec(shape=(None, None, 1), dtype=tf.float32, name='input_136'), name='input_136', description="created by layer 'input_136'")
y1: KerasTensor(type_spec=TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None), name='dense_176/BiasAdd:0', description="created by layer 'dense_176'")
y2: tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
outputs:  KerasTensor(type_spec=TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None), name='multiply_47/mul:0', description="created by layer 'multiply_47'")


In [77]:
input_a = tf.keras.Input(shape=(None,int(hyperparams['features_num'])))
input_b = tf.keras.Input(shape=(None,int(hyperparams['labels_num'])))
y1 = tf.constant([[2]])
y2 = tf.constant([[2,3,4,5]])
print(y1)
print(y2)
outputs = tf.keras.layers.Multiply()([y1,y2])
print(outputs)
#model = tf.keras.Model(inputs=[input_a, input_b], outputs=outputs)

tf.Tensor([[2]], shape=(1, 1), dtype=int32)
tf.Tensor([[2 3 4 5]], shape=(1, 4), dtype=int32)
tf.Tensor([[ 4  6  8 10]], shape=(1, 4), dtype=int32)
